<h1 style="background-color:orange;color:white;">Please Upvote it, If you find interesting!</h1>

In [ ]:
#import libraries
import numpy as np
import pandas as pd

In [ ]:
#reading train_csv file
train_set = pd.read_csv('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv')

In [ ]:
# train = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train/'

# from PIL import Image

# im = Image.open(train+'image1.jpg')
# width, height = im.size
# print(width,height)

In [ ]:
train_set.head()

In [ ]:
# we have 2 columns given image and it's repective class
train_set.columns

In [ ]:
#get some intution about classes of image
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#store frequency of image in given class
label_counts = train_set['Class'].value_counts()

#initalize plot figure of given size
plt.figure(figsize=(12,10))

#draw image of label_counts using seaborn barplot
sns.barplot(label_counts.index,label_counts.values)

#lable x-axis
plt.xlabel('Image Class', fontsize =12)

#lable y-axis
plt.ylabel('frequency', fontsize = 12)
plt.show()

In [ ]:
#to read images
import cv2

#use to show progress bar
from tqdm import tqdm

In [ ]:
# In this dataset we have different shape of image, we fix to 64*64
img_size = 128

def read_img(img_path):
    # cv2.IMREAD_COLOR to store image in color form(length*width*numberofchannels)
    img = cv2.imread(img_path,cv2.IMREAD_COLOR)
    #resize image
    img = cv2.resize(img, (img_size, img_size))
        
    return img

In [ ]:
#store image in form of list of matrices
train_img = []

# store path of training image folder
train_img_path = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train/'

for img_name in tqdm(train_set['Image'].values):
    train_img.append(read_img(train_img_path + img_name))

In [ ]:
# as pixel values is in between 0-255 so we divide by 255 so that range is in between 0-1
# as it easier for your cost function to find a local or global minimum and cnn train more efficiently

x_train = np.array(train_img,np.float32)/255

In [ ]:
x_train.shape

In [ ]:
# store classes in form of index
label_list = train_set['Class'].tolist()
label_numeric = {k: v+1 for v, k in enumerate(set(label_list))}
print(label_numeric)

print(label_list[0])

#store it in form of numpy array
y_train = [label_numeric[k] for k in label_list]
y_train = np.array(y_train)

In [ ]:
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime
import tensorflow as tf

In [ ]:
# convert all classes of data in form of category in binary form 
y_train = to_categorical(y_train)

In [ ]:
y_train[0]

Load Resnet model

In [ ]:
base_model = applications.ResNet50(weights='imagenet',
                                   include_top=False,
                                   input_shape=(img_size,img_size, 3))

In [ ]:
# base_model.trainable = True
# set_trainable = False
# for layer in base_model.layers:
#     if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False
# layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
# pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [ ]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
# add_model.add(Dropout(0.3))
# add_model.add(Dense(512, activation='relu'))
# add_model.add(Dropout(0.3))
# add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.4))
add_model.add(Dense(128, activation='relu'))
add_model.add(Dropout(0.4))
# add_model.add(Dense(64, activation='relu'))
# add_model.add(Dropout(0.4))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

In [ ]:
batch_size = 32      
epochs = 50                

# data augmentation
train_datagen = ImageDataGenerator(zoom_range=0.3, rotation_range=50,
 width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
 horizontal_flip=True, fill_mode="nearest")

train_datagen.fit(x_train)

In [ ]:

# chk_path = 'holiday.h5'
# log_dir = 'checkpoint/logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

# checkpoint = ModelCheckpoint(filepath=chk_path,
#                              save_best_only=True,
#                              verbose=1,
#                              mode='min',
#                              moniter='val_loss')

# earlystop = EarlyStopping(monitor='val_loss', 
#                           min_delta=0, 
#                           patience=3, 
#                           verbose=1, 
#                           restore_best_weights=True)
                        
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
#                               factor=0.2, 
#                               patience=6, 
#                               verbose=1, 
#                               min_delta=0.0001)

# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)
# csv_logger = CSVLogger('training_log')

# callbacks = [checkpoint,reduce_lr,csv_logger]

In [ ]:
# train model
history = model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                               steps_per_epoch=x_train.shape[0] // batch_size,
                               epochs=epochs)

In [ ]:
import os.path

test_images = os.listdir('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/')
test_img = []
for image in test_images:
    test_img.append(read_img('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/' + image))
    


In [ ]:
# store image in form of matrices and predict the test_img
x_test = np.array(test_img, np.float32) / 255
predictions = model.predict(x_test)

In [ ]:
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in label_numeric.items()}
pred_labels = [rev_y[k] for k in predictions]

In [ ]:
# store the prediction
sub = pd.DataFrame({'Image': test_images, 'Class': pred_labels})
sub.to_csv('submission.csv', index = False)
sub.head()

In [ ]:
import gc
gc.collect()